<a href="https://colab.research.google.com/github/tkdlqh2/wanted_8-/blob/main/3_NLU_%EB%AC%B8%EC%9E%A5%EC%9C%A0%EC%82%AC%EB%8F%84%EA%B3%84%EC%82%B0/NLU_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. 설치 및 설정

In [ ]:
!pip install transformers

In [ ]:
import os
import sys
import pandas as pd
import numpy as np 

import re
import requests

import torch
import torch.nn as nn
import tensorflow as tf

from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

In [ ]:
# GPU 메모리 에러 체크
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
# seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [164]:
train_df = pd.read_json('/content/drive/MyDrive/data/klue-sts-v1.1/klue-sts-v1.1_train.json')
dev_df = pd.read_json('/content/drive/MyDrive/data/klue-sts-v1.1/klue-sts-v1.1_dev.json')

### 1. 전처리

In [ ]:
train_df.head()

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0..."
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1..."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1..."


In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(train_df.head(1))
# 3.0 보다 평균 점수가 높으면 1 낮으면 0

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}","{'agreement': '0:0:0:2:5:0', 'annotators': ['07', '13', '15', '10', '12', '02', '19'], 'annotations': [3, 4, 4, 4, 3, 4, 4]}"


In [ ]:
train_df.shape

(11668, 6)

In [165]:
#f1 score로 메트릭 정할 경우 binary
#pearson으로 할경우 [0,5]

train_df['binary'] = [train_df['labels'][i]['binary-label'] for i in range(len(train_df))]

In [166]:
train_df.head()

,guid,source,sentence1,sentence2,labels,annotations,binary
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0...",1
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1...",0
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1...",0
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1...",0
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1...",1


#### 1-1. 띄어쓰기, 맞춤법 오류?
에어비앤비 리뷰는 맞춤법 오류가 많을 수 있음. 

In [ ]:
#pyKoSpace로 띄어쓰기
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
from pykospacing import Spacing
spacing = Spacing()

In [ ]:
#train_df['sentence1'].apply(spacing)

0                         숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.
1            위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.
2                  회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.
3        긴급 고용안정 지원금은 지역 고용대응 등 특별지원금, 지자체별 소상공인 지원사업, ...
4                              호스트의 답장이 늦으나, 개선될 것으로 보입니다.
                               ...                        
11663                             일단 방은 사진이랑 똑같고 매우 깨끗합니다.
11664                                 생일이 삼월이라는 친구 이름이 뭐야?
11665        그녀는 지속적으로 나에게 그녀가 만든 새로운 규칙을 매일 매일 이 메일로 보냈다.
11666                               개 1마리 고양이 3마리 너무 귀여워요!
11667                                학회 홍보 메일은 회신 메일을 보내지마
Name: sentence1, Length: 11668, dtype: object

In [ ]:
# 돌려봤는데, 얼추 봐선 띄어쓰기가 오히려 의미를 해칠 수도 있다고 판단되어 일단 저장만 해놓는걸로
train_df['sentence1']

0                         숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.
1            위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.
2                  회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.
3        긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...
4                              호스트의 답장이 늦으나, 개선될 것으로 보입니다.
                               ...                        
11663                             일단 방은 사진이랑 똑같고 매우 깨끗합니다.
11664                                 생일이 삼월이라는 친구 이름이 뭐야?
11665         그녀는 지속적으로 나에게 그녀가 만든 새로운 규칙을 매일 매일 이메일로 보냈다.
11666                               개 1마리 고양이 3마리 너무 귀여워요!
11667                                학회 홍보 메일은 회신 메일을 보내지마
Name: sentence1, Length: 11668, dtype: object

In [ ]:
# 맞춤법 오류
!pip install git+https://github.com/ssut/py-hanspell.git


In [68]:
from hanspell import spell_checker
result = spell_checker.check(u'안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.')
result.as_dict()['checked']

'안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.'

In [114]:
# 30분 걸림. 안하는게 나음
#sentence1
checked_lst = []
for i in range(len(train_df)):
  checked = spell_checker.check(train_df['sentence1'][i]).as_dict()['checked']
  checked_lst.append(checked)

In [132]:
# 특수문자 때문에 에러가 발생해서 일단 특문 클렌징 진행
temp = spell_checker.check(train_df['sentence2'][1308]).as_dict()

ParseError: ignored

In [137]:
def special_cleansing(text):
    text = re.sub('[-=+#/\:^$@*\"※~&%ㆍ』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', text)
    return text

In [138]:
train_df['sentence2'] = train_df['sentence2'].apply(special_cleansing)

In [139]:
train_df['sentence2'][1308]

'국가 RD 우수성과 FDA 승인 뇌병증 치료제 세노바마테 및 지능형 소프트웨어 닥터S 등 상위 100위까지 선정되었습니다.'

In [140]:
#sentence2
checked_lst2 = []
temp_lst = []
for i in range(len(train_df)):
  temp = spell_checker.check(train_df['sentence2'][i]).as_dict()
  checked = temp['checked']
  err = temp['errors']
  checked_lst2.append(checked)
  temp_lst.append(err)

In [150]:
import numpy as np
np.mean(temp_lst)
#평균 0.56개의 맞춤법 에러 > 1개도 안고침;

0.5634213232773397

In [167]:
train_df['spell_checked1'] = checked_lst
train_df['spell_checked2'] = checked_lst2
train_df.head()

,guid,source,sentence1,sentence2,labels,annotations,binary,spell_checked1,spell_checked2
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0...",1,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1...",0,위반행위 조사 등을 거부·방해·기피한 자는 500만 원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산뿐만이 아니었다.
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1...",0,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1...",0,"긴급 고용안정 지원금은 지역 고용 대응 등 특별 지원금, 지자체별 소상공인 지원 사...","고용보험이 1차 고용안전망이라면, 국민 취업지원 제도는 2차 고용안전망입니다."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1...",1,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.


#### 1-2 숫자,영어,특문 제거

 별도로 불용어를 제거하지말고 토큰화 단계에서 조사, 접속사를 제거하자.




In [155]:
def cleansing(text):
    text = re.sub('[0-9a-zA-Z-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', text)
    return text

In [169]:
train_df['spell_checked1'] = train_df['spell_checked1'].apply(cleansing)
train_df['spell_checked2'] = train_df['spell_checked2'].apply(cleansing)

In [170]:
train_df.head()

,guid,source,sentence1,sentence2,labels,annotations,binary,spell_checked1,spell_checked2
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0...",1,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1...",0,위반행위 조사 등을 거부방해기피한 자는 만 원 이하 과태료 부과 대상이다,시민들 스스로 자발적인 예방 노력을 한 것은 아산뿐만이 아니었다
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1...",0,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1...",0,긴급 고용안정 지원금은 지역 고용 대응 등 특별 지원금 지자체별 소상공인 지원 사업...,고용보험이 차 고용안전망이라면 국민 취업지원 제도는 차 고용안전망입니다
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1...",1,호스트의 답장이 늦으나 개선될 것으로 보입니다,호스트 응답이 늦었지만 개선될 것으로 보입니다


#### 1-3 형태소분리(khaiii)


In [ ]:
!git clone https://github.com/kakao/khaiii.git 
!pip install cmake 
!mkdir build 
!cd build && cmake /content/khaiii 
!cd /content/build/ && make all 
!cd /content/build/ && make resource 
!cd /content/build && make install 
!cd /content/build && make package_python 
!pip install /content/build/package_python


In [89]:
from khaiii import KhaiiiApi 
api = KhaiiiApi()

In [171]:
# 조사, 접속사 제거 유무 결정하기
# 출처 - https://sy-log.tistory.com/58 (품사리스트)
# 의존명사, 주격조사, 보격조사, 관형격조사, 목적격조사, 부사격조사, 호격조사, 인용격조사, 보조사, 접속조사 제거
stop_tags = ['NNB', 'JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC']

In [109]:
# 예시
khaii_lst = []
test_sents = train_df['sentence1'][:5]
for sent in test_sents:
  morph_list = []  
  for word in api.analyze(sent):
    for morph in word.morphs:
      if morph.tag not in stop_tags: # 이부분 제외하면 조사+접속사 제거하지 않고 출력
        morph_list.append(morph.lex)
  text = ' '.join(morph_list)
  khaii_lst.append(text)
khaii_lst

['숙소 위치 찾 기 쉽 고 일반 적 이 ㄴ 한국 반지하 숙소 이 ㅂ니다 .',
 '위반 행위 조사 거부 · 방해 · 기피 하 ㄴ 500 만 이하 과태료 부과 대상 이 다 .',
 '회사 보내 ㄴ 메일 이 지메일 아니 라 다른 지메일 계정 전달 하 여 주 어 .',
 '긴급 고용 안정 지원금 지역 고용 대응 특별 지원금 , 지자 체 별 소상 공 이 ㄴ 지원 사업 , 취업 성 공패키지 , 청년구직 활동지원금 , 길 ㄴ 급 복지 지원제도 지원금 중복 수급 불 가능 하 다 .',
 '호스트 답장 늦 으나 , 개선 되 ㄹ 보이 ㅂ니다 .']

In [176]:
khaii_lst = []
test_sents = train_df['spell_checked1']
for sent in test_sents:
  morph_list = []  
  for word in api.analyze(sent):
    for morph in word.morphs:
      if morph.tag not in stop_tags: # 이부분 제외하면 조사+접속사 제거하지 않고 출력
        morph_list.append(morph.lex)
  text = ' '.join(morph_list)
  khaii_lst.append(text)
khaii_lst[:5]

['숙소 위치 찾 기 쉽 고 일반 적 이 ㄴ 한국 반지하 숙소 이 ㅂ니다',
 '위반 행위 조사 거부 방해기피 하 ㄴ 만 이하 과태료 부과 대상 이 다',
 '회사 보내 ㄴ 메일 이 지메일 아니 라 다른 지메일 계정 전달 하 여 주 어',
 '긴급 고용 안정 지원금 지역 고용 대응 특별 지원금 지자 체 별 소상 공 이 ㄴ 지원 사업 취업 성 공패키지 청년 구직 활동 지원금 길 ㄴ 급 복지 지원 제도 지원금 중복 수급 불 가능 하 다',
 '호스트 답장 늦 으나 개선 되 ㄹ 보이 ㅂ니다']

In [175]:
khaii_lst2 = []
test_sents = train_df['spell_checked2']
for sent in test_sents:
  morph_list = []  
  for word in api.analyze(sent):
    for morph in word.morphs:
      if morph.tag not in stop_tags: # 이부분 제외하면 조사+접속사 제거하지 않고 출력
        morph_list.append(morph.lex)
  text = ' '.join(morph_list)
  khaii_lst2.append(text)
khaii_lst2[:5]

['숙박 시설 위치 쉽 게 찾 을 있 고 한국 대표 적 이 ㄴ 반지하 숙박 시설 이 ㅂ니다',
 '시민 들 스스로 자발 적 이 ㄴ 예방 노력 \xa0 하 ㄴ 아 산 아니 었 다',
 '사람 들 주로 네이버 메일 쓰 는 이유 알리 어 주 어',
 '고용 보험 차 아 고용 안 전망 이 라면 국민 취업 지원 제도 차 고용 안 전망 이 ㅂ니다',
 '호스트 응답 늦 었 지만 개선 되 ㄹ 보이 ㅂ니다']

In [177]:
train_df['khaii1'] = khaii_lst
train_df['khaii2'] = khaii_lst2
train_df.head()

,guid,source,sentence1,sentence2,labels,annotations,binary,spell_checked1,spell_checked2,khaii1,khaii2
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0...",1,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다,숙소 위치 찾 기 쉽 고 일반 적 이 ㄴ 한국 반지하 숙소 이 ㅂ니다,숙박 시설 위치 쉽 게 찾 을 있 고 한국 대표 적 이 ㄴ 반지하 숙박 시설 이 ㅂ니다
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1...",0,위반행위 조사 등을 거부방해기피한 자는 만 원 이하 과태료 부과 대상이다,시민들 스스로 자발적인 예방 노력을 한 것은 아산뿐만이 아니었다,위반 행위 조사 거부 방해기피 하 ㄴ 만 이하 과태료 부과 대상 이 다,시민 들 스스로 자발 적 이 ㄴ 예방 노력 하 ㄴ 아 산 아니 었 다
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1...",0,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,회사 보내 ㄴ 메일 이 지메일 아니 라 다른 지메일 계정 전달 하 여 주 어,사람 들 주로 네이버 메일 쓰 는 이유 알리 어 주 어
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1...",0,긴급 고용안정 지원금은 지역 고용 대응 등 특별 지원금 지자체별 소상공인 지원 사업...,고용보험이 차 고용안전망이라면 국민 취업지원 제도는 차 고용안전망입니다,긴급 고용 안정 지원금 지역 고용 대응 특별 지원금 지자 체 별 소상 공 이 ㄴ 지...,고용 보험 차 아 고용 안 전망 이 라면 국민 취업 지원 제도 차 고용 안 전망 이...
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1...",1,호스트의 답장이 늦으나 개선될 것으로 보입니다,호스트 응답이 늦었지만 개선될 것으로 보입니다,호스트 답장 늦 으나 개선 되 ㄹ 보이 ㅂ니다,호스트 응답 늦 었 지만 개선 되 ㄹ 보이 ㅂ니다


고민할 지점
1. 형태소 분리 전처리 사용할 것인가? 하지않고 그대로 사전학습 모델 토크나이저 사용할 것인가?
2. 형태소 분리 사용한다면 조사 제거할것인가?


In [185]:
cd drive/MyDrive/data

/content/drive/MyDrive/data


In [186]:
train_df.to_csv('klue_sts_preprocessed.csv', index=False)

#### 센텐스피스


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.8 MB/s 


In [ ]:
import sentencepiece as spm
import csv

In [ ]:
cd '/content/drive/MyDrive/data/klue-sts-v1.1'

/content/drive/MyDrive/data/klue-sts-v1.1


In [ ]:
with open('klue_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['sentence1']))

In [ ]:
spm.SentencePieceTrainer.Train('--input=klue_review.txt \
--model_prefix=klue --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [ ]:
vocab_lst = pd.read_csv('klue.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_lst[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,니다,0
4,▁이,-1
5,▁있,-2
6,▁지,-3
7,습니다,-4
8,에서,-5
9,▁가,-6


In [ ]:
vocab_lst.sample(10)

,0,1
317,▁요,-314
2160,정책,-2157
4693,몸,-4690
2662,▁거리입니다,-2659
4797,괄,-4794
3031,▁이용하는,-3028
2498,▁건강보험,-2495
772,▁그래,-769
3249,▁사람들과,-3246
2156,운영,-2153


In [ ]:
sp = spm.SentencePieceProcessor()
vocab_file = "klue.model"
sp.load(vocab_file)

True

In [ ]:
lines = [
  "숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.",
  "호스트 응답이 늦었지만 개선될 것으로 보입니다",
]
for line in lines:
  print(line)
  print(' '.join(sp.encode_as_pieces(line)))
  print(sp.encode_as_ids(line))
  print()

숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.
▁숙소 ▁위치는 ▁찾기 ▁쉽 고 ▁일반 적인 ▁한국의 ▁반 지 하 ▁숙소입니다 .
[26, 821, 2947, 2845, 3890, 984, 313, 3233, 402, 3880, 3883, 432, 3875]

호스트 응답이 늦었지만 개선될 것으로 보입니다
▁호스트 ▁응답 이 ▁늦 었 지만 ▁개선 될 ▁것으로 ▁보입니다
[67, 1232, 3877, 358, 3989, 152, 941, 4226, 535, 1948]

